In [ ]:
# Part 2

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Load your trained GAN model
# Replace 'your_model_path' with the path to your saved generator model
generator = keras.models.load_model('your_model_path')

# Define a function to perform Layer-wise Relevance Propagation (LRP)
def lrp(model, image, relevance_scaling=1.0):
    # Create a copy of the model with custom LRP Relevance Layer
    lrp_model = keras.models.clone_model(model)

    # Replace the final activation layer with a linear activation layer
    lrp_model.add(layers.Activation('linear'))

    # Compile the LRP model
    lrp_model.compile(optimizer='adam', loss='mse')

    # Perform LRP
    relevance_input = np.ones_like(image)
    relevance_map = lrp_model.predict(image) * relevance_scaling
    relevance_input = lrp_model.layers[-2].backward(relevance_map, relevance_input)

    return relevance_input

# Generate random noise as input to the generator
noise = np.random.normal(0, 1, size=(1, 100))

# Generate a fake image using the generator
generated_image = generator.predict(noise)

# Perform LRP on the generator
relevance_input = lrp(generator, noise)

# Visualize the generated image and the corresponding relevance scores
plt.subplot(1, 2, 1)
plt.imshow(generated_image[0])
plt.title('Generated Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(relevance_input[0])
plt.title('Relevance Scores')
plt.axis('off')

plt.show()


